# Best Score -> 0.39729


> Indented block



## Import the needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler #you can use minmax scaler too
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import TomekLinks
from xgboost import XGBClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported fro

## Import the training data

In [132]:
train_data = pd.read_csv("/content/train_dataset.csv")

FileNotFoundError: ignored

In [3]:
train_data.tail()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_diff,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_diff,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,next_month_plan
10495,1438655,4G,Smartphone,MALE,Kalutara,40-50,64,2088.13,479,0.346959,226.626969,226.280010,15.972982,30,41.785388,318,43.433333,0,43.433333,12,26.519444,16.035284,102,29.866667,0.0,7,14.571429,8.017054,0,0,NaN,NaN,PKG6
10496,7827264,4G,Smartphone,FEMALE,Colombo,<20,6,2053.50,23168,0.219985,2525.781404,2525.561419,926.754902,25,683.641213,0,0.000000,0,0.000000,0,NaN,0.000000,32,25.366667,0.0,3,10.861111,5.151029,0,0,580.0,3.0,PKG1
10497,1433957,4G,Smartphone,FEMALE,Colombo,30-40,69,2827.33,4553,4.831711,581.854178,577.022468,151.773327,30,161.529659,0,0.000000,0,0.000000,0,NaN,0.000000,0,0.000000,0.0,0,NaN,0.000000,0,1,NaN,NaN,PKG6
10498,8494507,4G,Smartphone,MALE,Kandy,60-70,233,645.43,2069,4.722031,191.290240,186.568210,68.986909,30,53.804147,0,0.000000,0,0.000000,0,NaN,0.000000,105,18.650000,0.0,18,5.887963,4.956822,0,1,NaN,NaN,PKG1
10499,3569439,3G,Smartphone,MALE,Puttalam,40-50,33,498.20,536,0.003048,177.700564,177.697516,17.877463,30,43.567639,0,0.000000,0,0.000000,0,NaN,0.000000,15,4.366667,0.0,7,2.228571,1.096541,0,0,NaN,NaN,PKG1


## Encoding Labels

In [4]:
train_data['next_month_plan'] = train_data['next_month_plan'].astype('category')

In [5]:
train_data['next_month_plan'].unique()

['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']
Categories (8, object): ['PKG2', 'PKG1', 'PKG6', 'PKG4', 'PKG5', 'PKG3', 'PKG8', 'PKG7']

In [6]:
labels_x = ['PKG0','PKG1', 'PKG2', 'PKG3', 'PKG4', 'PKG5', 'PKG6', 'PKG7', 'PKG8']

In [7]:
le = preprocessing.LabelEncoder()

In [8]:
le.fit(labels_x)

LabelEncoder()

In [9]:
le.classes_

array(['PKG0', 'PKG1', 'PKG2', 'PKG3', 'PKG4', 'PKG5', 'PKG6', 'PKG7',
       'PKG8'], dtype='<U4')

In [10]:
train_data['encoded_class_labels'] = le.transform(train_data['next_month_plan'])

In [11]:
train_data = train_data.drop(columns=['next_month_plan'])

In [12]:
train_data.head(20)

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_diff,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_diff,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count,encoded_class_labels
0,6875768,4G,Smartphone,MALE,Kegalle,40-50,219,761.62,6317,2.773958,622.685848,619.911890,210.579939,30,197.116822,0,0.000000,0,0.000000,0,NaN,0.000000,677,152.866667,0.0,27,25.099383,34.759221,0,1,NaN,NaN,2
1,6406277,4G,Smartphone,MALE,Galle,40-50,36,2482.52,28365,3.079438,2569.429138,2566.349700,945.531235,30,573.299639,0,0.000000,0,0.000000,0,NaN,0.000000,76,35.433333,0.0,15,5.071111,6.779644,0,0,600.0,2.0,1
2,3563570,4G,Smartphone,MALE,Kandy,30-40,186,1565.55,18581,48.193713,2739.515326,2691.321613,619.378811,30,619.750500,0,0.000000,0,0.000000,0,NaN,0.000000,0,0.000000,0.0,0,NaN,0.000000,0,0,0.0,1.0,6
3,1853666,4G,Smartphone,FEMALE,Colombo,40-50,24,2397.49,91071,22.437842,7523.355693,7500.917850,3035.708609,30,2174.203127,0,0.000000,0,0.000000,0,NaN,0.000000,0,0.000000,0.0,0,NaN,0.000000,0,0,1110.0,2.0,4
4,2794331,4G,Smartphone,FEMALE,Kandy,50-60,37,2501.75,19736,53.302092,1239.220170,1185.918078,657.889802,30,329.152516,0,0.000000,0,0.000000,0,NaN,0.000000,258,87.616667,0.0,23,11.256522,16.224071,0,1,1350.0,2.0,4
5,3619017,2G,Basic,MALE,Galle,40-50,30,495.54,0,0.001531,0.173772,0.172241,0.020356,30,0.034965,0,0.000000,0,0.000000,0,NaN,0.000000,2,1.066667,0.0,3,0.700000,0.233432,0,0,NaN,NaN,1
6,7633257,4G,Smartphone,MALE,Kegalle,30-40,35,1977.19,14782,205.394300,1001.028832,795.634533,492.737055,30,177.211955,0,0.000000,0,0.000000,0,NaN,0.000000,0,0.000000,0.0,0,NaN,0.000000,0,0,110.0,1.0,5
7,5507217,4G,Smartphone,MALE,Gampaha,30-40,170,2925.00,3473,6.426329,1132.109515,1125.683187,115.785698,30,222.353815,43,9.966667,0,9.966667,7,6.147619,2.939495,0,0.000000,0.0,0,NaN,0.000000,0,0,650.0,1.0,5
8,7566055,4G,Smartphone,MALE,Kalutara,20-30,48,473.59,3870,0.000000,1180.032728,1180.032728,552.976737,7,337.334503,7,6.700000,0,6.700000,2,3.575000,1.313412,34,22.883333,0.0,3,11.338889,4.661264,0,0,110.0,1.0,2
9,7324903,4G,Smartphone,MALE,Colombo,20-30,81,1324.60,20876,3.450117,1290.141176,1286.691059,695.869196,30,334.779967,7,7.400000,0,7.400000,1,7.400000,1.351049,11,6.816667,0.0,6,1.922222,1.310881,0,0,360.0,2.0,2


## Handling Missing Values

In [13]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           10500 non-null  int64  
 1   device_type                  10477 non-null  object 
 2   device_category              10470 non-null  object 
 3   gender                       10365 non-null  object 
 4   district_name                10500 non-null  object 
 5   age_group                    10365 non-null  object 
 6   network_stay                 10500 non-null  int64  
 7   average_monthly_bill_amount  10500 non-null  float64
 8   dusage_sum                   10500 non-null  int64  
 9   dusage_min                   10500 non-null  float64
 10  dusage_max                   10500 non-null  float64
 11  dusage_diff                  10500 non-null  float64
 12  dusage_avg                   9642 non-null   float64
 13  dusage_days     

In [14]:
train_data['device_category'].unique()

array(['Smartphone', 'Basic', 'Feature phone', 'Pluggable card', 'Tablet',
       nan, 'Modem'], dtype=object)

In [15]:
train_data['device_type'] = train_data['device_type'].fillna(value = 'Unknown')
train_data['device_category'] = train_data['device_category'].fillna(value = 'Unknown')
train_data['gender'] = train_data['gender'].fillna(value = 'Unknown')
train_data['age_group'] = train_data['age_group'].fillna(value = 'Unknown')

In [16]:
train_data['dusage_avg'] = train_data['dusage_avg'].fillna(value = 0)
train_data['vusage_offnet_avg'] = train_data['vusage_offnet_avg'].fillna(value = 0)
train_data['add_on_tot_rental'] = train_data['add_on_tot_rental'].fillna(value = 0)
train_data['add_on_count'] = train_data['add_on_count'].fillna(value = 0)
train_data['vusage_onnet_avg'] = train_data['vusage_onnet_avg'].fillna(value = 0)

In [17]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10500 entries, 0 to 10499
Data columns (total 33 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           10500 non-null  int64  
 1   device_type                  10500 non-null  object 
 2   device_category              10500 non-null  object 
 3   gender                       10500 non-null  object 
 4   district_name                10500 non-null  object 
 5   age_group                    10500 non-null  object 
 6   network_stay                 10500 non-null  int64  
 7   average_monthly_bill_amount  10500 non-null  float64
 8   dusage_sum                   10500 non-null  int64  
 9   dusage_min                   10500 non-null  float64
 10  dusage_max                   10500 non-null  float64
 11  dusage_diff                  10500 non-null  float64
 12  dusage_avg                   10500 non-null  float64
 13  dusage_days     

In [18]:
train_data = train_data.drop(columns=['network_stay','dusage_min','dusage_max','vusage_onnet_max','vusage_onnet_min'])

## Creating pre-processing pipeline

In [19]:
train_data.columns

Index(['primary_identifier', 'device_type', 'device_category', 'gender',
       'district_name', 'age_group', 'average_monthly_bill_amount',
       'dusage_sum', 'dusage_diff', 'dusage_avg', 'dusage_days',
       'dusage_stddev', 'vusage_onnet_sum', 'vusage_onnet_diff',
       'vusage_onnet_days', 'vusage_onnet_avg', 'vusage_onnet_stddev',
       'vusage_offnet_sum', 'vusage_offnet_max', 'vusage_offnet_min',
       'vusage_offnet_days', 'vusage_offnet_avg', 'vusage_offnet_stddev',
       'number_of_fixed_bb_accounts', 'number_of_iptv_accounts',
       'add_on_tot_rental', 'add_on_count', 'encoded_class_labels'],
      dtype='object')

In [20]:
data_cat_cols = ['device_type', 'device_category', 'gender', 'district_name', 'age_group']

In [30]:
data_quan_cols = ['average_monthly_bill_amount', 'dusage_sum', 
                   'dusage_diff','dusage_avg', 'dusage_days', 
                  'dusage_stddev', 'vusage_onnet_sum','vusage_onnet_diff', 
                  'vusage_onnet_days', 'vusage_onnet_avg',
                  'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
                  'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
                  'vusage_offnet_stddev', 'number_of_fixed_bb_accounts','number_of_iptv_accounts', 
                  'add_on_tot_rental', 'add_on_count']

In [31]:
X = train_data.drop(columns=['encoded_class_labels', 'primary_identifier'])

#### Dropping columns selected by Feature selection

In [ ]:
columns_remove = ['network_stay', 'dusage_max']

In [ ]:
# for col in columns_remove:
#     data_quan_cols.remove(col)

In [ ]:
#X = X.drop(columns=columns_remove)

In [32]:
X.columns

Index(['device_type', 'device_category', 'gender', 'district_name',
       'age_group', 'average_monthly_bill_amount', 'dusage_sum', 'dusage_diff',
       'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum',
       'vusage_onnet_diff', 'vusage_onnet_days', 'vusage_onnet_avg',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
       'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count'],
      dtype='object')

In [33]:
y = train_data['encoded_class_labels']

In [34]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, random_state=10, test_size = 0.05, stratify = y)

In [35]:
X_train.head(20)

,device_type,device_category,gender,district_name,age_group,average_monthly_bill_amount,dusage_sum,dusage_diff,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_diff,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
6155,4G,Smartphone,MALE,Colombo,60-70,831.92,34164,1363.552058,1138.819100,30,399.148743,0,0.000000,0,0.000000,0.000000,5,3.600000,0.0,5,1.103333,0.672225,1,2,360.0,2.0
2288,4G,Smartphone,MALE,Kandy,20-30,2677.35,11984,1053.219714,399.470436,30,281.884382,0,0.000000,0,0.000000,0.000000,0,0.000000,0.0,0,0.000000,0.000000,0,0,0.0,0.0
9714,4G,Smartphone,MALE,Colombo,40-50,2627.77,19830,1014.929033,661.027294,30,220.638181,0,0.000000,0,0.000000,0.000000,0,0.000000,0.0,0,0.000000,0.000000,0,0,0.0,0.0
7157,4G,Smartphone,MALE,Colombo,40-50,4795.76,8498,712.027700,283.283955,30,216.864356,0,0.000000,0,0.000000,0.000000,114,17.050000,0.0,22,5.197727,4.522210,0,0,360.0,2.0
738,4G,Smartphone,MALE,Colombo,50-60,3978.40,36917,2074.850735,1230.593121,30,466.919448,0,0.000000,0,0.000000,0.000000,0,0.000000,0.0,0,0.000000,0.000000,0,0,1500.0,1.0
490,4G,Smartphone,FEMALE,Colombo,40-50,2582.98,6820,750.020165,227.350520,30,193.195645,0,0.000000,0,0.000000,0.000000,330,66.483333,0.0,18,18.367593,18.339099,0,1,650.0,1.0
467,4G,Smartphone,MALE,Kalutara,40-50,906.76,7652,957.962963,255.090212,30,282.895663,0,0.000000,0,0.000000,0.000000,104,17.700000,0.0,27,3.873457,4.288491,0,0,110.0,2.0
6161,4G,Smartphone,MALE,Colombo,30-40,2148.93,10470,1624.676157,349.017543,30,339.574431,323,43.866667,25,12.956000,12.350004,53,21.683333,0.0,9,5.916667,4.704471,0,0,0.0,0.0
1343,4G,Smartphone,MALE,Gampaha,40-50,2759.05,15901,1131.000009,530.056132,30,268.249199,66,20.950000,8,8.258333,5.098354,45,8.533333,0.0,11,4.140909,2.426823,0,0,0.0,0.0
5792,4G,Smartphone,MALE,Colombo,50-60,1069.44,13193,1681.258982,439.789331,30,321.920352,0,0.000000,0,0.000000,0.000000,31,13.333333,0.0,8,3.879167,2.785113,1,0,600.0,2.0


In [36]:
X_train['age_group'].unique()

array(['60-70', '20-30', '40-50', '50-60', '30-40', '>70', 'Unknown',
       '<20'], dtype=object)

In [37]:
X_train.shape

(9975, 26)

In [38]:
quan_pipeline = Pipeline([
    ('std_scaler', StandardScaler())
])

quan_transformed = quan_pipeline.fit_transform(X_train[data_quan_cols])

In [39]:
data_pipeline = ColumnTransformer([
    ('numerical', quan_pipeline, data_quan_cols),
    ('categorical', OneHotEncoder(handle_unknown = 'ignore'), data_cat_cols),
    
])

train_data_processed = data_pipeline.fit_transform(X_train)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

num_attribs = list(housing_num)
cat_attribs = list(housing_cat)

num_pipeline = Pipeline([
         
         ('attribs_adder', FunctionTransformer(add_extra_features, validate=False)),
         ('imputer', SimpleImputer(strategy="median")),
         ('std_scaler', StandardScaler())
    ])
full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs),
        (cat_attribs),
    ])

housing_prepared = full_pipeline.fit_transform(housing_date1)

In [40]:
train_data_processed[0]

array([-0.84948216,  1.34223453, -0.00532918,  1.2518874 ,  0.53621625,
        0.16788833, -0.18788699, -0.2600062 , -0.28080456, -0.27565714,
       -0.24650206, -0.47954691, -0.5031088 , -0.06635089, -0.42126523,
       -0.4872795 , -0.53776094,  4.00769298,  3.49200503,  0.0538823 ,
        0.60012268,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        1.        ,  0.        ,  0.        ,  0.        ])

In [41]:
X_test.head()

,device_type,device_category,gender,district_name,age_group,average_monthly_bill_amount,dusage_sum,dusage_diff,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_diff,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
231,4G,Smartphone,FEMALE,Colombo,30-40,1716.27,12647,1875.519728,665.666485,19,592.455861,0,0.0,0,0.0,0.0,13,10.366667,0.0,4,3.395833,1.946967,0,0,250.0,1.0
9966,4G,Smartphone,MALE,Ratnapura,20-30,985.32,1089,267.741177,37.566127,29,65.361534,0,0.0,0,0.0,0.0,30,5.850000,0.0,11,2.775758,1.919775,0,0,0.0,0.0
486,4G,Smartphone,MALE,Gampaha,40-50,3518.42,22732,1351.566897,783.872928,29,352.976495,0,0.0,0,0.0,0.0,113,44.533333,0.0,4,28.425000,10.514607,0,0,760.0,2.0
9686,4G,Smartphone,MALE,Colombo,30-40,2559.86,14666,1064.386329,488.870862,30,249.683006,0,0.0,0,0.0,0.0,0,0.000000,0.0,0,0.000000,0.000000,0,0,0.0,0.0
6310,4G,Smartphone,FEMALE,Trincomalee,30-40,1802.86,31567,1150.825547,1052.261866,30,317.321847,0,0.0,0,0.0,0.0,52,24.683333,0.0,12,4.397222,4.734033,0,0,450.0,2.0


In [42]:
X_train.columns

Index(['device_type', 'device_category', 'gender', 'district_name',
       'age_group', 'average_monthly_bill_amount', 'dusage_sum', 'dusage_diff',
       'dusage_avg', 'dusage_days', 'dusage_stddev', 'vusage_onnet_sum',
       'vusage_onnet_diff', 'vusage_onnet_days', 'vusage_onnet_avg',
       'vusage_onnet_stddev', 'vusage_offnet_sum', 'vusage_offnet_max',
       'vusage_offnet_min', 'vusage_offnet_days', 'vusage_offnet_avg',
       'vusage_offnet_stddev', 'number_of_fixed_bb_accounts',
       'number_of_iptv_accounts', 'add_on_tot_rental', 'add_on_count'],
      dtype='object')

In [43]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 525 entries, 231 to 229
Data columns (total 26 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   device_type                  525 non-null    object 
 1   device_category              525 non-null    object 
 2   gender                       525 non-null    object 
 3   district_name                525 non-null    object 
 4   age_group                    525 non-null    object 
 5   average_monthly_bill_amount  525 non-null    float64
 6   dusage_sum                   525 non-null    int64  
 7   dusage_diff                  525 non-null    float64
 8   dusage_avg                   525 non-null    float64
 9   dusage_days                  525 non-null    int64  
 10  dusage_stddev                525 non-null    float64
 11  vusage_onnet_sum             525 non-null    int64  
 12  vusage_onnet_diff            525 non-null    float64
 13  vusage_onnet_days 

In [44]:
test_data_processed = data_pipeline.transform(X_test)

In [45]:
test_data_processed

array([[-0.24995533, -0.01257233,  0.28025057, ...,  0.        ,
         0.        ,  0.        ],
       [-0.74548779, -0.74031586, -0.61658146, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.97177507,  0.62242452, -0.0120146 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.09860764, -0.80680636, -0.76582097, ...,  0.        ,
         0.        ,  0.        ],
       [-0.31827705, -0.75807185, -0.66589201, ...,  0.        ,
         1.        ,  0.        ],
       [-0.92795926, -0.74037883, -0.69109345, ...,  0.        ,
         0.        ,  0.        ]])

### Loading Submission data

In [77]:
submission_data = pd.read_csv("/content/test_dataset_v2.csv")

In [78]:
submission_data.head()

,primary_identifier,device_type,device_category,gender,district_name,age_group,network_stay,average_monthly_bill_amount,dusage_sum,dusage_min,dusage_max,dusage_diff,dusage_avg,dusage_days,dusage_stddev,vusage_onnet_sum,vusage_onnet_max,vusage_onnet_min,vusage_onnet_diff,vusage_onnet_days,vusage_onnet_avg,vusage_onnet_stddev,vusage_offnet_sum,vusage_offnet_max,vusage_offnet_min,vusage_offnet_days,vusage_offnet_avg,vusage_offnet_stddev,number_of_fixed_bb_accounts,number_of_iptv_accounts,add_on_tot_rental,add_on_count
0,2003793,4G,Smartphone,MALE,Colombo,40-50,4,689.42,148,0.000000,148.146634,148.146634,37.037382,4,38.251243,0,0.0,0.0,0.0,0,NaN,0.0,0,0.000000,0.0,0,NaN,0.000000,0,1,NaN,NaN
1,1776101,4G,Smartphone,MALE,Colombo,40-50,7,799.93,300,0.180222,119.466825,119.286604,10.027957,30,24.125172,0,0.0,0.0,0.0,0,NaN,0.0,107,16.266667,0.0,21,5.107143,4.924111,0,0,NaN,NaN
2,6945050,2G,Basic,MALE,Colombo,20-30,20,1538.45,8215,0.000000,4048.511104,4048.511104,2053.950266,4,932.358667,0,0.0,0.0,0.0,0,NaN,0.0,108,90.000000,0.0,8,13.558333,16.397071,0,0,110.0,2.0
3,2472049,4G,Smartphone,FEMALE,Matara,40-50,10,995.75,11613,0.000000,1057.448432,1057.448432,430.118721,27,268.457304,0,0.0,0.0,0.0,0,NaN,0.0,0,0.416667,0.0,2,0.333333,0.087355,0,0,NaN,NaN
4,7197266,4G,Smartphone,MALE,Kandy,30-40,154,553.61,26505,28.515345,2490.832742,2462.317397,883.505624,30,660.353290,0,0.0,0.0,0.0,0,NaN,0.0,0,0.000000,0.0,0,NaN,0.000000,0,0,350.0,1.0


## Handling missing values in submission data

In [79]:
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           4500 non-null   int64  
 1   device_type                  4487 non-null   object 
 2   device_category              4484 non-null   object 
 3   gender                       4445 non-null   object 
 4   district_name                4500 non-null   object 
 5   age_group                    4445 non-null   object 
 6   network_stay                 4500 non-null   int64  
 7   average_monthly_bill_amount  4500 non-null   float64
 8   dusage_sum                   4500 non-null   int64  
 9   dusage_min                   4500 non-null   float64
 10  dusage_max                   4500 non-null   float64
 11  dusage_diff                  4500 non-null   float64
 12  dusage_avg                   4136 non-null   float64
 13  dusage_days       

In [80]:
submission_data['device_category'].unique()

array(['Smartphone', 'Basic', 'Feature phone', 'M2M', nan, 'Tablet',
       'Pluggable card', 'Modem'], dtype=object)

In [81]:
submission_data['device_type'] = submission_data['device_type'].fillna(value = 'Unknown')
submission_data['device_category'] = submission_data['device_category'].fillna(value = 'Unknown')
submission_data['gender'] = submission_data['gender'].fillna(value = 'Unknown')
submission_data['age_group'] = submission_data['age_group'].fillna(value = 'Unknown')

In [ ]:
#submission_data = submission_data.drop(columns = ['vusage_onnet_avg'])

In [ ]:
#submission_data = submission_data.drop(columns = columns_remove)

In [82]:
submission_data['dusage_avg'] = submission_data['dusage_avg'].fillna(value = 0)
submission_data['vusage_offnet_avg'] = submission_data['vusage_offnet_avg'].fillna(value = 0)
submission_data['add_on_tot_rental'] = submission_data['add_on_tot_rental'].fillna(value = 0)
submission_data['add_on_count'] = submission_data['add_on_count'].fillna(value = 0)
submission_data['vusage_onnet_avg'] = submission_data['vusage_onnet_avg'].fillna(value = 0)

In [83]:
submission_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 32 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   primary_identifier           4500 non-null   int64  
 1   device_type                  4500 non-null   object 
 2   device_category              4500 non-null   object 
 3   gender                       4500 non-null   object 
 4   district_name                4500 non-null   object 
 5   age_group                    4500 non-null   object 
 6   network_stay                 4500 non-null   int64  
 7   average_monthly_bill_amount  4500 non-null   float64
 8   dusage_sum                   4500 non-null   int64  
 9   dusage_min                   4500 non-null   float64
 10  dusage_max                   4500 non-null   float64
 11  dusage_diff                  4500 non-null   float64
 12  dusage_avg                   4500 non-null   float64
 13  dusage_days       

In [84]:
prim_id = submission_data['primary_identifier']

In [88]:
submission_data = submission_data.drop(columns=['network_stay','dusage_min','dusage_max','vusage_onnet_max','vusage_onnet_min'])

In [89]:
submission_data.shape

(4500, 27)

In [90]:
submission_X = submission_data.drop(columns=['primary_identifier'])

In [91]:
submission_X.shape

(4500, 26)

## Pre-processing the submission data

In [92]:
submission_data_processed = data_pipeline.transform(submission_X)

In [93]:
submission_data_processed[0].shape

(69,)

## Model Code

In [46]:
Y_train.unique()

array([1, 6, 8, 3, 7, 5, 4, 2])

In [47]:
unique, counts = np.unique(Y_train, return_counts=True)

In [48]:
dict(zip(unique, counts))

{1: 3490, 2: 1286, 3: 1631, 4: 698, 5: 1480, 6: 703, 7: 346, 8: 341}

In [49]:
3490*0.13

453.7

In [50]:
strategy = {
    1: 3490,
    2: 1250,
    3: 1600,
    4: 698,
    5: 1450,
    6: 700,
    7: 346,
    8: 341
}

In [51]:
undersample = NearMiss(sampling_strategy=strategy,n_neighbors=2)

In [52]:
X_near , y_near = undersample.fit_resample(train_data_processed,Y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [53]:
X_near.shape

(9875, 69)

In [54]:
unique, counts = np.unique(y_near, return_counts=True)
dict(zip(unique, counts))

{1: 3490, 2: 1250, 3: 1600, 4: 698, 5: 1450, 6: 700, 7: 346, 8: 341}

### Oversampling

In [55]:
strategy_os = {
    1: 3490,
    2: 1250,
    3: 1600,
    4: 698,
    5: 1450,
    6: 700,
    7: 700,
    8: 700
}

In [56]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42, sampling_strategy = strategy_os)
X_res, y_res = sm.fit_resample(X_near, y_near)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [57]:
X_res.shape

(10588, 69)

In [58]:
unique, counts = np.unique(y_res, return_counts=True)
dict(zip(unique, counts))

{1: 3490, 2: 1250, 3: 1600, 4: 698, 5: 1450, 6: 700, 7: 700, 8: 700}

clf_xgb = XGBClassifier()
param_dist = {'n_estimators': [100, 300, 500, 700, 900],
              'learning_rate': [1, 0.1, 0.01, 0.02],
              'max_depth': [3, 4, 5, 6],
             }

clf = GridSearchCV(clf_xgb, 
                   param_dist,
                   cv = 5,
                   verbose = 3, 
                   n_jobs = -1)

clf.fit(X_res, y_res)

clf.best_params_
Best parameters = {'learning_rate': 0.02, 'max_depth': 5, 'n_estimators': 900}

clf.best_score_

In [128]:
model = XGBClassifier(label_encoder = False, 
                      learning_rate = 0.01,max_depth = 9,
                       n_estimators = 900, subsample = 0.5349338392246493)
#model = XGBClassifier(label_encoder = False)
#900 gave the best score
#max depth 7

In [129]:
model.fit(X_res, y_res)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              label_encoder=False, learning_rate=0.01, max_delta_step=0,
              max_depth=9, min_child_weight=1, missing=None, n_estimators=900,
              n_jobs=1, nthread=None, objective='multi:softprob',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
              seed=None, silent=None, subsample=0.5349338392246493,
              verbosity=1)

In [130]:
y_pred = model.predict(test_data_processed)

In [131]:
# Getting f1 score
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.71      0.83      0.77       183
           2       0.56      0.15      0.23        68
           3       0.44      0.66      0.53        86
           4       0.33      0.14      0.19        37
           5       0.67      0.78      0.72        78
           6       0.52      0.30      0.38        37
           7       0.27      0.17      0.21        18
           8       0.42      0.61      0.50        18

    accuracy                           0.59       525
   macro avg       0.49      0.45      0.44       525
weighted avg       0.58      0.59      0.56       525



In [115]:
# Getting f1 score
from sklearn.metrics import classification_report
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           1       0.73      0.83      0.78       183
           2       0.71      0.18      0.28        68
           3       0.41      0.66      0.51        86
           4       0.42      0.14      0.20        37
           5       0.67      0.78      0.72        78
           6       0.47      0.22      0.30        37
           7       0.25      0.17      0.20        18
           8       0.47      0.78      0.58        18

    accuracy                           0.59       525
   macro avg       0.52      0.47      0.45       525
weighted avg       0.60      0.59      0.56       525



In [116]:
y_submission = model.predict(submission_data_processed)

In [117]:
unique, counts = np.unique(y_submission, return_counts=True)
dict(zip(unique, counts))

{1: 1810, 2: 183, 3: 1029, 4: 138, 5: 839, 6: 214, 7: 113, 8: 174}

In [118]:
y_submission

array([2, 1, 3, ..., 3, 7, 3])

In [119]:
y_submissions_series = pd.Series(y_submission)

In [120]:
submission_data = {'primary_identifier' : pd.to_numeric(prim_id),
                 'next_month_plan' : pd.to_numeric(y_submissions_series)}

submission_df = pd.DataFrame(submission_data)

In [121]:
submission_df.shape

(4500, 2)

In [122]:
submission_df.to_csv("submission_6.csv")

In [123]:
submission_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   primary_identifier  4500 non-null   int64
 1   next_month_plan     4500 non-null   int64
dtypes: int64(2)
memory usage: 70.4 KB


In [1]:
from keras.layers import Input,Dense,Conv1D,MaxPool1D,GlobalMaxPooling1D,Add,GlobalAveragePooling1D,Reshape,multiply
from keras.layers.merge import concatenate
from keras.models import Model
from keras import callbacks

ModuleNotFoundError: No module named 'keras'

In [105]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
#from catboost import Pool, CatBoostClassifier
import itertools
import pickle, gzip
import glob

In [110]:
#K.clear_session()
def build_model(dropout_rate=0.25,activation='relu'):
    start_neurons = 256
    feat_ip = Input(shape=(train_data.shape[1],), name='feature_ip')
    
    x = BatchNormalization()(feat_ip)
    x = Dense(start_neurons, activation=activation)(x)    
    feat1 = Dropout(rate=dropout_rate)(x)

    x = BatchNormalization()(feat1)
    x = Dense(start_neurons//2, activation=activation)(x)    
    feat2 = Dropout(rate=dropout_rate)(x)
    
    x = BatchNormalization()(feat2)
    x = Dense(start_neurons//4, activation=activation)(x)    
    feat3 = Dropout(rate=dropout_rate)(x)
    x = BatchNormalization()(feat3)
    x = Dense(start_neurons//8, activation=activation)(x)    
    feat4 = Dropout(rate=dropout_rate/2)(x)

    feat_concat = concatenate([feat1,feat2,feat3,feat4])
    out = Dense(len(classes), activation='softmax')(feat_concat)

    model = Model(inputs=[feat_ip], outputs=[out])    
    return model

In [111]:
model = build_model(dropout_rate=0.25,activation='relu')

NameError: ignored

In [108]:
model.compile(loss= keras.losses.SparseCategoricalCrossentropy(),
             optimizer=tf.keras.optimizers.Adam(lr=0.001),
             metrics=['accuracy'])
history = model.fit(X_train, Y_train,
                    validation_data=[X_test, Y_test], 
                    epochs=250,
                    batch_size=200)

AttributeError: ignored